In [20]:
%config Completer.use_jedi = False

In [21]:
import pandas as pd
import pickle
from tqdm.notebook import tqdm
import numpy as np
import re
import string

from libs.LastNamesInference import LastNamesInference

In [22]:
address_clean = pd.read_pickle('/data/datasets/WOS/US/address_clean.p')

In [23]:
addres_raw = pd.read_table('/data/datasets/WOS/US/Adresses_race.txt',delimiter='\t')

In [24]:
print("{} authors (raw)".format(addres_raw.cluster_id.unique().shape[0]))

4713444 authors (raw)


In [25]:
print("{} papers (raw)".format(addres_raw.id_art.unique().shape[0]))

5431451 papers (raw)


In [26]:
print("{} institutions (raw)".format(addres_raw.Institution.unique().shape[0]))

261336 institutions (raw)


In [27]:
print("{} authors".format(address_clean.cluster_id.unique().shape[0]))

3441264 authors


In [28]:
print("{} papers".format(address_clean.id_art.unique().shape[0]))

4553335 papers


In [29]:
print("{} universities".format(address_clean.carnegie_id.unique().shape[0]))

685 universities


In [30]:
4553335/5431451

0.8383275482002875

In [31]:
3441264/4713444

0.7300954461323822

In [32]:
def npapers(c,v):
    df = address_clean[address_clean[c]==v]
    print("{} papers".format(df.id_art.unique().shape[0]))
    print("{} authors".format(df.cluster_id.unique().shape[0]))
    print("{} universities".format(df.carnegie_id.unique().shape[0]))

In [33]:
npapers('hbcu','HBCU')

35518 papers
24829 authors
62 universities


In [34]:
npapers('hsi','HSI')

278109 papers
169469 authors
127 universities


In [35]:
npapers('womens','Womens')

8710 papers
5320 authors
25 universities


In [36]:
address_clean.groupby('selindex')['id_art','cluster_id','carnegie_id'].nunique()

/tmp/ipykernel_136/252003900.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  address_clean.groupby('selindex')['id_art','cluster_id','carnegie_id'].nunique()


,id_art,cluster_id,carnegie_id
selindex,,,
inclusive,262617,168731,196
more_selective,3264304,2280454,187
not_indexed,428957,303370,96
selective,1283167,833853,206


In [37]:
address_clean.groupby('avg_citations_Q')['id_art','cluster_id','carnegie_id'].nunique()

/tmp/ipykernel_136/3887997064.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  address_clean.groupby('avg_citations_Q')['id_art','cluster_id','carnegie_id'].nunique()


,id_art,cluster_id,carnegie_id
avg_citations_Q,,,
"(0.1, 1.47)",1746770,1204592,547
"(1.48, 1.74)",1815227,1189355,78
"(1.77, 4.07)",1826346,1215155,60


In [38]:
address_clean.groupby('avg_citations_Q10')['id_art','cluster_id','carnegie_id'].nunique()

/tmp/ipykernel_136/2565696184.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  address_clean.groupby('avg_citations_Q10')['id_art','cluster_id','carnegie_id'].nunique()


,id_art,cluster_id,carnegie_id
avg_citations_Q10,,,
"(0.1, 1.24)",578602,393312,430
"(1.24, 1.34)",558175,354724,61
"(1.34, 1.43)",581558,365063,40
"(1.43, 1.54)",500969,302332,33
"(1.54, 1.59)",599016,366844,22
"(1.59, 1.66)",590696,369040,23
"(1.66, 1.81)",672059,407611,23
"(1.82, 1.91)",682885,431533,11
"(1.91, 2.15)",651981,392126,17


In [39]:
address_clean.groupby('usnr_rank_cat')['id_art','cluster_id','carnegie_id'].nunique()

/tmp/ipykernel_136/4243000484.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  address_clean.groupby('usnr_rank_cat')['id_art','cluster_id','carnegie_id'].nunique()


,id_art,cluster_id,carnegie_id
usnr_rank_cat,,,
top_10,960994,592170,12
top_100,2517913,1684950,89
not_top,1901865,1330966,584


In [40]:
address_clean.columns

Index(['cluster_id', 'id_art', 'grid_id', 'carnegie_id', 'carnegie_name', 'r1',
       'control', 'obereg', 'hbcu', 'tribal', 'hsi', 'msi', 'womens',
       'selindex', 'n_papers_on_WOS', 'ncitable_docs', 'Top10CR', 'Top5CR',
       'Top1CR', 'Top10CR_Q', 'Top5CR_Q', 'Top1CR_Q', 'npapers', 'impact',
       'avg_citations', 'npapers_Q', 'impact_Q', 'avg_citations_Q',
       'usnr_rank', 'usnr_rank_cat', 'avg_citations_Q10'],
      dtype='object')

In [41]:
address_clean[(address_clean.hsi=='HSI') & (address_clean.usnr_rank_cat =='top_10')]

,cluster_id,id_art,grid_id,carnegie_id,carnegie_name,r1,control,obereg,hbcu,tribal,...,Top1CR_Q,npapers,impact,avg_citations,npapers_Q,impact_Q,avg_citations_Q,usnr_rank,usnr_rank_cat,avg_citations_Q10


In [42]:
address_clean[(address_clean.hsi=='HSI') & (address_clean.usnr_rank_cat =='top_100')].carnegie_name.value_counts()

University of California, Santa Barbara    71406
University of California, Riverside        58756
Texas A & M University-College Station       812
Name: carnegie_name, dtype: int64

In [43]:
address_clean[(address_clean.hbcu=='HBCU') & (address_clean.usnr_rank_cat =='top_100')].carnegie_name.value_counts()

Howard University    9768
Name: carnegie_name, dtype: int64

In [44]:
address_clean[(address_clean.womens=='Womens') & (address_clean.usnr_rank_cat =='top_100')].carnegie_name.value_counts()

Series([], Name: carnegie_name, dtype: int64)

In [45]:
harvard = address_clean[address_clean.carnegie_name =='Harvard University']

In [46]:
address_clean[address_clean.usnr_rank==2].carnegie_name.value_counts()

Harvard University                             721009
Columbia University                            206912
MIT                                            177157
Columbia University in the City of New York     26920
Massachusetts Institute of Technology           21250
Name: carnegie_name, dtype: int64

In [47]:
avg_citations_df = address_clean[['carnegie_name','avg_citations']].drop_duplicates()

In [48]:
def min_max_mean_papers_per_inst(group_df):
    inst_n = group_df.carnegie_id.value_counts()
    min_inst_n = min(inst_n)
    max_inst_n = max(inst_n)
    mean_inst_n = np.mean(inst_n)
    res = 'min: {}, max: {}, mean: {}'.format(min_inst_n,max_inst_n,min_inst_n)
    return(res)

In [49]:
group_df = address_clean[address_clean.usnr_rank_cat=='not_top']

In [50]:
inst_n = group_df.carnegie_id.value_counts()

In [51]:
address_clean.groupby('usnr_rank_cat').apply(min_max_mean_papers_per_inst)

usnr_rank_cat
top_10     min: 78923, max: 721009, mean: 78923
top_100        min: 812, max: 314933, mean: 812
not_top            min: 1, max: 251607, mean: 1
dtype: object

In [52]:
inst_n = group_df.carnegie_id.value_counts()

In [53]:
address_clean.groupby('usnr_rank_cat').apply(lambda x: min_max_mean_papers_per_inst(x))

usnr_rank_cat
top_10     min: 78923, max: 721009, mean: 78923
top_100        min: 812, max: 314933, mean: 812
not_top            min: 1, max: 251607, mean: 1
dtype: object

## Number of papers by institution and flags

In [54]:
n_inst = address_clean.groupby('carnegie_id')['id_art'].nunique()

In [55]:
metadata_df = address_clean.drop_duplicates('carnegie_id')

In [56]:
n_inst = n_inst.to_frame().rename(columns={'id_art':'n_papers'}).merge(metadata_df, on='carnegie_id')

In [57]:
n_inst = n_inst.drop(columns=['cluster_id','id_art'])

In [58]:
n_inst.sort_values('n_papers').to_excel('../../data/n_papers_by_institution.xlsx')

In [59]:
n_inst.columns

Index(['carnegie_id', 'n_papers', 'grid_id', 'carnegie_name', 'r1', 'control',
       'obereg', 'hbcu', 'tribal', 'hsi', 'msi', 'womens', 'selindex',
       'n_papers_on_WOS', 'ncitable_docs', 'Top10CR', 'Top5CR', 'Top1CR',
       'Top10CR_Q', 'Top5CR_Q', 'Top1CR_Q', 'npapers', 'impact',
       'avg_citations', 'npapers_Q', 'impact_Q', 'avg_citations_Q',
       'usnr_rank', 'usnr_rank_cat', 'avg_citations_Q10'],
      dtype='object')

Methods HBCU, HSI, WDC

In [60]:
hbcu_df = address_clean[address_clean.hbcu=='HBCU']
hsi_df = address_clean[address_clean.hsi=='HSI']
wc_df = address_clean[address_clean.womens=='Womens']

In [61]:
hbcu_n = hbcu_df.groupby('carnegie_id').id_art.nunique().sort_values()
hsi_n = hsi_df.groupby('carnegie_id').id_art.nunique().sort_values()
wc_n = wc_df.groupby('carnegie_id').id_art.nunique().sort_values()

In [62]:
hbcu_n[-10:].sum()/hbcu_n.sum()

0.6978785062059161

In [63]:
hbcu_n[:40].sum()/hbcu_n.sum()

0.09904860845316857

In [64]:
hsi_n[-10:].sum()/hsi_n.sum()

0.7309611393596027

In [65]:
hsi_n[:-10].sum()/hsi_n.sum()

0.2690388606403973

In [66]:
wc_n[-10:].sum()/wc_n.sum()

0.9059078467153284

In [67]:
wc_n[:-10].sum()/wc_n.sum()

0.09409215328467153